In [1]:
import os, sys, ipaddress, random
from datetime import datetime
parentDir = os.path.abspath('..')
sys.path.append(parentDir)
from netLookup import netLookup

In [2]:
def buildNetworks(v4: int, v6: int=0) -> list:
    networks=[]
    for _ in range(0, v4):
        # Do Stuff
        ip = random.randrange(0, 2**32)
        ip = ipaddress.ip_address(ip)
        subnet = random.randrange(8, 25)
        network = ipaddress.ip_network(f'{ip}/{subnet}', strict=False)
        networks.append(network)
    return networks

def buildQueryIps(v4: int, v6: int=0) -> list:
    ips = []
    for _ in range(0, v4):
        ip = ipaddress.ip_address(random.randrange(0,2**32))
        ips.append(ip)
    return ips

In [3]:
%%timeit
buildNetworks(1000)

4.48 ms ± 163 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [4]:
%%timeit
buildQueryIps(10000)

6.47 ms ± 104 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


During initial testing this ran around 4.49ms with 1000 networks generated and 6.42ms with 10000 ips generated.

In [12]:
table = netLookup.Table()
start = datetime.now()
numberOfNetworks = 10000
networks = buildNetworks(numberOfNetworks)
for network in networks:
    #print(hash(network))
    entry = {
        'network': network,
        'data': network
        }
    table.add(entry)
end = datetime.now()
print(f'table build {end - start}')
numberOfQueries = 1000
ips = buildQueryIps(numberOfQueries)
notFound = 0
found = 0
start = datetime.now()
for ip in ips:
    try:
        data = table.query(ip)        
        found += 1
    except ValueError:
        notFound += 1
        pass
end = datetime.now()
print(f'table query: {end - start}')
print(f'''Networks found: {found}
Networks not found: {notFound}
Percentage found: {(found/numberOfQueries)*100}''')
        

table build 0:00:00.093135
table query: 0:00:01.869529
Networks found: 990
Networks not found: 10
Percentage found: 99.0


Using 10000 networks and 1000 queries approximatly 99% of queried IP's were found in about 2 seconds. During these loops the average table build time was around .05 seconds and the lookup time was around 1.7 seconds and scaled nearly linerly when the number of IP queries changed.